# WEB SCRAPING IMAGES FROM GOOGLE

Came across two methods:
1. Using Beautiful Soup
2. Using Selenium

Got stuck on Beautiful Soup so had to switch to Selenium. I found Selenium a bit more complex than Beautiful Soup, but I think it's visual output feature makes it a fascinating tool.

The Selenium package does what a human user would normally do on the web browser. In this case I want to go to Google Images and search for images of Dogs and store it on my desktop, Selenium would automate the process for us by specifying which image you want to download and how many images you want to download. 

### Importing libraries

In [1]:
import selenium
from selenium import webdriver
import time
import os
import requests
from PIL import Image
import io
import hashlib

### Chrome Driver Path

To use Selenium with Google Chrome we need to download a Chrome Driver, depending on the Google Chrome Version the Chrome Driver is installed. 

In [2]:
DRIVER_PATH =  '/Users/apurvasalvi/Desktop/GauguinBot/chromedriver'
wd = webdriver.Chrome(executable_path=DRIVER_PATH)
wd.get('https://google.com') 
search_box = wd.find_element_by_css_selector('input.gLFyf') #input box selector
search_box.send_keys('dogs')
wd.quit()

The above lines of code only opens the browser and gives the input query and quits.

The second phase would involve to search for the query, go to the image section and get the respective image links using css selectors. 

The third phase of the code will be to download the images from the link onto your local computer. 

### Code for Web Scraping from Google Images

In [3]:
def fetch_image_urls(plot:str, max_links_to_fetch:int, wd:webdriver):
    def scroll_to_end(wd):
        wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(1)
    # build the google plot
    search_url = "https://www.google.com/search?safe=off&site=&tbm=isch&source=hp&q={q}&oq={q}&gs_l=img"
    
    wd.get(search_url.format(q=plot))
    
    image_urls = set() #duplicates won't be added
    image_count = 0
    results_start = 0
    while image_count < max_links_to_fetch:
        scroll_to_end(wd)
        
        #find elements based on the tag and class name using css selector
        thumbnail_results = wd.find_elements_by_css_selector("img.Q4LuWd") 
        number_links = len(thumbnail_results)
        
        print(f"Found: {number_links} search results. Extracting links from {results_start}:{number_links}")
        
        for img in thumbnail_results[results_start:number_links]:
            #try to click every thumbnail such that we can get the real image behind it
            try:
                img.click()
                time.sleep(1)
            except Exception:
                continue
            
            #extract image urls    
            actual_images = wd.find_elements_by_css_selector('img.n3VNCb')
            for actual_image in actual_images:
                if actual_image.get_attribute('src') and 'http' in actual_image.get_attribute('src'):
                    image_urls.add(actual_image.get_attribute('src'))
            
            image_count = len(image_urls)
            
            if len(image_urls) >= max_links_to_fetch:
                print(f"Found: {len(image_urls)} image links, done!")
                break
        else:
            print("Found:", len(image_urls), "image links, looking for more ...")
            time.sleep(10)
            show_more_results = wd.find_element_by_css_selector(".mye4qd")
            if show_more_results:
                wd.execute_script("document.querySelector('.mye4qd').click();")

        # move the result startpoint further down
        results_start = len(thumbnail_results)
    
    return image_urls

def download_image(folder_path:str,file_name:str,url:str):
    try:
        image_content = requests.get(url).content
    except Exception as e:
        print(f"ERROR - Could not download {url} - {e}")
        
    try:
        image_file = io.BytesIO(image_content)
        image = Image.open(image_file).convert('RGB')  #Opens and identifies the given image file
        folder_path = os.path.join(folder_path,file_name) #Joins 2 or more pathname components
        if os.path.exists(folder_path):
            #if the path exists, add file to the folder path
            file_path = os.path.join(folder_path,hashlib.sha1(image_content).hexdigest()[:10] + '.jpg')
        else:
            #else create a new folder and add file to the new folder
            os.mkdir(folder_path)
            file_path = os.path.join(folder_path,hashlib.sha1(image_content).hexdigest()[:10] + '.jpg')
        with open(file_path, 'wb') as f:  #'wb': mode for binary random access, opens and truncates the file to 0 bytes
            image.save(f, "JPEG", quality=85)  
        print(f"SUCCESS - saved {url} - as {file_path}")
    except Exception as e:
        print(f"ERROR - Could not save {url} - {e}")

if __name__ == '__main__':
    wd = webdriver.Chrome(executable_path=DRIVER_PATH)  #controls chrome driver and allows you to drive the browser
    plot_names = ["heatmap"]  #list of search keywords
    for plot in plot_names:
        wd.get('https://google.com')   #loads webpage in the current browser session
        search_box = wd.find_element_by_css_selector('input.gLFyf')   #finds an element by css selector and returns it if found
        search_box.send_keys(plot)   #simulates typing into the element
        links = fetch_image_urls(plot,1000,wd)  #gets image urls
        images_path = '/Users/apurvasalvi/Desktop/GauguinBot/images' #folder to save the element
        for i in links:
            download_image(images_path,plot,i)   #downloads images to the specified path
    wd.quit()

Found: 100 search results. Extracting links from 0:100
Found: 178 image links, looking for more ...
Found: 312 search results. Extracting links from 100:312
Found: 557 image links, looking for more ...
Found: 412 search results. Extracting links from 312:412
Found: 725 image links, looking for more ...
Found: 612 search results. Extracting links from 412:612
Found: 1001 image links, done!
SUCCESS - saved https://www.grapecity.com/componentone/docs/win/online-flexchart/images/discrete-heatmap.png - as /Users/apurvasalvi/Desktop/GauguinBot/images/heatmap/f91186019e.jpg
ERROR - Could not save https://policyviz.com/wp-content/uploads/2017/04/m25OG.png - cannot identify image file <_io.BytesIO object at 0x111eadfc0>
SUCCESS - saved https://www.jefftk.com/us-contra-heatmap.jpg - as /Users/apurvasalvi/Desktop/GauguinBot/images/heatmap/45b5b3c69b.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcSO7gvne7zJ2ctl6YejNzk1cHnQDE9qy-AtPA&usqp=CAU - as /Users/apurvasalvi/Des

/anaconda3/lib/python3.7/site-packages/PIL/Image.py:2655: UserWarning: image file could not be identified because WEBP support not installed
  warnings.warn(message)


ERROR - Could not save https://www.formget.com/wp-content/uploads/2018/03/Heatmap-WordPress-Plugings.png - cannot identify image file <_io.BytesIO object at 0x112b32410>
SUCCESS - saved https://website-assets-fw.freshworks.com/attachments/cjq4q3bq303bxhqfzw7rhbvia-heatmaps-dynamic-scr.one-half.png - as /Users/apurvasalvi/Desktop/GauguinBot/images/heatmap/4df6d60154.jpg
SUCCESS - saved https://docs.losant.com/images/dashboards/heatmap-normal.png - as /Users/apurvasalvi/Desktop/GauguinBot/images/heatmap/ec86c15bd0.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcSGsE77L5fjTI8wcBjVJ3-tR9M7lLHSuF43JQ&usqp=CAU - as /Users/apurvasalvi/Desktop/GauguinBot/images/heatmap/d267156b6e.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcRHxfGtrqg7UKVSTAUR9YgyHf9Wj89RJWqcDQ&usqp=CAU - as /Users/apurvasalvi/Desktop/GauguinBot/images/heatmap/731cac09b1.jpg
SUCCESS - saved https://sessioncam.com/wp-content/uploads/2018/03/heatmap-browser-scrollreach.jp

SUCCESS - saved https://static.wingify.com/gcp/uploads/sites/3/2020/01/Heatmap-free-tools.jpg - as /Users/apurvasalvi/Desktop/GauguinBot/images/heatmap/fae9e57b43.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcSNVjlTWXBTznLMoQDDjF6w2UONB6KVxYtnVg&usqp=CAU - as /Users/apurvasalvi/Desktop/GauguinBot/images/heatmap/966ae8c172.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcS5-T-IZv57cQCvGs2R4WVX_Brte8Zu86z4_Q&usqp=CAU - as /Users/apurvasalvi/Desktop/GauguinBot/images/heatmap/7d33b5caae.jpg
SUCCESS - saved https://www.visualcinnamon.com/img/blog/2013/heatmap-in-r/r_heatmap_feature.png - as /Users/apurvasalvi/Desktop/GauguinBot/images/heatmap/27a1b47b18.jpg
SUCCESS - saved https://sparxsystems.com/enterprise_architect_user_guide/15.1/images/heatmap.png - as /Users/apurvasalvi/Desktop/GauguinBot/images/heatmap/b51e8db4e4.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcSR7k0hi77x-r4pApyIYCoYCnFoyb704VguXQ&usq

SUCCESS - saved https://neliosoftware.com/wp-content/uploads/2014/05/heatmaps-screen.png - as /Users/apurvasalvi/Desktop/GauguinBot/images/heatmap/bc0cb6b0c1.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcTT0MbJ2vXYkfsqEOPiZeFeAnHkFzf7gRuKbg&usqp=CAU - as /Users/apurvasalvi/Desktop/GauguinBot/images/heatmap/a35cf9e0c7.jpg
SUCCESS - saved https://www.elegantthemes.com/blog/wp-content/uploads/2020/01/Heatmaps-Featured-Image.jpg - as /Users/apurvasalvi/Desktop/GauguinBot/images/heatmap/f923ed8d0c.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcSD9ptcEQVB_jfbFOs2fs3EOBG1tcvSzTi5FA&usqp=CAU - as /Users/apurvasalvi/Desktop/GauguinBot/images/heatmap/b5f4acb935.jpg
SUCCESS - saved https://lh3.googleusercontent.com/proxy/UvZntFvQ5uvIpogKpfkrukzg6e9nrgNqdPWYfBsWQCToJyNPh50t06usbu38xzyDCPmTRs3_ZLgu78-ABhbKMqBvuU6LSD8HDfOAXjZyAHPJStO0ZqxX_SmKYN8h1ne6ecl-fg - as /Users/apurvasalvi/Desktop/GauguinBot/images/heatmap/ea3d85ccab.jpg
SUCCESS - sav

...

SUCCESS - saved https://docs.hytrust.com/CloudControl/5.4.0/Online/Content/Resources/Images/LV4-HeatMap.png - as /Users/apurvasalvi/Desktop/GauguinBot/images/heatmap/1b38edb922.jpg
SUCCESS - saved https://se.mathworks.com/help/examples/graphics/win64/CreateHeatmapFromMatrixDataExample_01.png - as /Users/apurvasalvi/Desktop/GauguinBot/images/heatmap/61f8113d1a.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcSycmah6D1AuYSqhyke_yPTKCx8WukwGJbjkg&usqp=CAU - as /Users/apurvasalvi/Desktop/GauguinBot/images/heatmap/a5bb172970.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcQ_3qNakXODrlbK1Fxi2bIijDB2q84xyeJjBA&usqp=CAU - as /Users/apurvasalvi/Desktop/GauguinBot/images/heatmap/5c85d0053d.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcTAkqaclkOmgYBuqO7C2PWQub2_AZC-uvE_Zg&usqp=CAU - as /Users/apurvasalvi/Desktop/GauguinBot/images/heatmap/dbf0625905.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=t

SUCCESS - saved https://www.researchgate.net/profile/Mehdi_Mirsaeidi/publication/313145263/figure/fig1/AS:616382920142866@1523968548392/Showing-a-false-color-heatmap-of-the-distances-between-arrays-The-color-scale-is-chosen.png - as /Users/apurvasalvi/Desktop/GauguinBot/images/heatmap/056ae8614b.jpg
ERROR - Could not save https://mouseflow.com/wp-content/uploads/2019/04/Screenshot1.png - cannot identify image file <_io.BytesIO object at 0x112b89eb8>
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcQSDgsH9cJLJr2rrveVKVYZC-n0fDUescxeVQ&usqp=CAU - as /Users/apurvasalvi/Desktop/GauguinBot/images/heatmap/a73895641e.jpg
ERROR - Could not save https://i.vimeocdn.com/video/762475534.webp?mw=1000&mh=562&q=70 - cannot identify image file <_io.BytesIO object at 0x1120e4ca8>
SUCCESS - saved https://learn.g2.com/hs-fs/hubfs/Capturly%20X%20G2%20What%20Can%20a%20Heatmap%20Tell%20You%20About%20Your%20Product%20page%3F-1.png?width=600&name=Capturly%20X%20G2%20What%20Can%20a%20Hea

SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcSkdUia3z6mtZVlexFvdwRhmdLmWJyTFX4_vA&usqp=CAU - as /Users/apurvasalvi/Desktop/GauguinBot/images/heatmap/c5c9d355ea.jpg
SUCCESS - saved https://www.slideteam.net/media/catalog/product/cache/960x720/h/e/heatmap_tracking_Slide01.jpg - as /Users/apurvasalvi/Desktop/GauguinBot/images/heatmap/78c3a6ee2f.jpg
SUCCESS - saved https://static.wingify.com/gcp/uploads/sites/3/2020/02/Heatmap-Visualization.jpg - as /Users/apurvasalvi/Desktop/GauguinBot/images/heatmap/cac894987d.jpg
SUCCESS - saved https://seaborn.pydata.org/_images/seaborn-heatmap-12.png - as /Users/apurvasalvi/Desktop/GauguinBot/images/heatmap/99383ecb85.jpg
SUCCESS - saved https://d2mvzyuse3lwjc.cloudfront.net/doc/en/Tutorial/images/Heat_Map_and_Virtual_Matrix_Manager/Heatmap-21.png?v=32493 - as /Users/apurvasalvi/Desktop/GauguinBot/images/heatmap/85a43281db.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcQzNnND17770tKKLKouv0HI1eTSV

SUCCESS - saved https://www.grapecity.com/componentone/docs/win/online-flexchart/images/gradient-heatmap.png - as /Users/apurvasalvi/Desktop/GauguinBot/images/heatmap/7321f402ec.jpg
SUCCESS - saved https://api.contentstack.io/v2/uploads/58e4e4c10884fe765eaf6faf/download?uid=blte800c5667d5f32f5 - as /Users/apurvasalvi/Desktop/GauguinBot/images/heatmap/39b98d311a.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcSXbnXjFtallyxURWqyKwHWm3gp1e7geH0yFw&usqp=CAU - as /Users/apurvasalvi/Desktop/GauguinBot/images/heatmap/a0431aecaf.jpg
SUCCESS - saved https://i0.wp.com/texblog.org/Wordpress/wp-content/uploads/2013/06/heatmap-100x1.png?resize=640%2C635 - as /Users/apurvasalvi/Desktop/GauguinBot/images/heatmap/248147cc41.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcRvUnbrPzBnsHWDrBMThXgOizJ1jH-Dmiqtow&usqp=CAU - as /Users/apurvasalvi/Desktop/GauguinBot/images/heatmap/946dff6c6f.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q

SUCCESS - saved https://docs.servicenow.com/bundle/orlando-performance-analytics-and-reporting/page/use/performance-analytics/image/heatmap-pivot-widget.png - as /Users/apurvasalvi/Desktop/GauguinBot/images/heatmap/ba0ae6d8f8.jpg
SUCCESS - saved https://miro.medium.com/max/916/1*j6cHz7EAN-P0d1gpQDpK6w.png - as /Users/apurvasalvi/Desktop/GauguinBot/images/heatmap/d9690854c7.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcTzcCvk8dTK89OgQ4g4eCy35UazDbkq549TFA&usqp=CAU - as /Users/apurvasalvi/Desktop/GauguinBot/images/heatmap/b8d08ccbed.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcRRf_uCswQyQQsAUnN35k-IfFf-MEvC6OD2_w&usqp=CAU - as /Users/apurvasalvi/Desktop/GauguinBot/images/heatmap/2073cb4b32.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcT9dRmqvZ1gL2dhfGKep6oZ9x-ieyGkH3-wQQ&usqp=CAU - as /Users/apurvasalvi/Desktop/GauguinBot/images/heatmap/7c981b939b.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.

### Contribution

External Source: 70%
Personal Contribution: 30%

### Citations

1. Article title:	Web Scraping Images from Google

   Website title:	Medium
   
   URL          :	https://medium.com/@wwwanandsuresh/web-scraping-images-from-google-9084545808a2
   
   
2. Article title:	Msalmannasir/Google_image_scraper

   Website title:	GitHub
   
   URL          :	https://github.com/Msalmannasir/Google_image_scraper/blob/master/google_img.

### Conclusion

This task helped me learn how to use Selenium to web scrape any image from the internet. The task is divided into 3 steps: Opening the Web Browser using Selenium, Getting the URLs, and Downloading the images using these URLs. The above lines of codes can be used to download any number of images from the internet. Thus, I have automated the process of getting images from the web using Selenium. 